<a href="https://colab.research.google.com/github/Stefano0210/IULM_DDM2324_Notebooks/blob/main/22_RegressioneLogisticaSemplice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Prepariamo l'ambiente per analisi successive scaricando un dataset e fornendo un'anteprima dei dati per confermare il successo delle operazioni.

In [ ]:
import pandas as pd
import statsmodels.api as sm


!wget "https://github.com/Stefano0210/IULM_DDM2324_Notebooks/raw/main/data/SubscriptionByAge.xlsx"
# Caricare il dataset (da sostituire con il percorso corretto del file)
data = pd.read_excel("SubscriptionByAge.xlsx")

data.rename(columns={"Subscribe?":"Subscribed"}, inplace=True)

data.head()




--2024-04-09 14:55:25--  https://github.com/Stefano0210/IULM_DDM2324_Notebooks/raw/main/data/SubscriptionByAge.xlsx
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Stefano0210/IULM_DDM2324_Notebooks/main/data/SubscriptionByAge.xlsx [following]
--2024-04-09 14:55:26--  https://raw.githubusercontent.com/Stefano0210/IULM_DDM2324_Notebooks/main/data/SubscriptionByAge.xlsx
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9304 (9.1K) [application/octet-stream]
Saving to: ‘SubscriptionByAge.xlsx’

SubscriptionByAge.x 100%[===================>]   9.09K  --.-KB/s    in 0s      

2024-04-09 14:55:26 (53.4 

Age  Subscribed
0   20           0
1   23           0
2   24           0
3   25           0
4   25           1

Creiamo una tabella che contenga i risultati ottenuti dalla regressione logistica.

In [ ]:
independent_vars = ['Intercept', 'Age']
data['Intercept'] = 1

logit_model = sm.Logit(data['Subscribed'], data[independent_vars])

fit = logit_model.fit()

print(fit.summary())

Optimization terminated successfully.
         Current function value: 0.472562
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:             Subscribed   No. Observations:                   41
Model:                          Logit   Df Residuals:                       39
Method:                           MLE   Df Model:                            1
Date:                Tue, 09 Apr 2024   Pseudo R-squ.:                  0.1494
Time:                        14:55:47   Log-Likelihood:                -19.375
converged:                       True   LL-Null:                       -22.777
Covariance Type:            nonrobust   LLR p-value:                  0.009096
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -5.6616      2.017     -2.807      0.005      -9.615      -1.708
Age            0.1281      0.

REBALANCE WITH SMOTE

Eseguiamo un'analisi di regressione logistica su un dataset bilanciato attraverso SMOTE per prevedere la probabilità che un utente sottoscriva in base alla sua età. Questo approccio aiuta a mitigare i potenziali bias dovuti a un dataset sbilanciato e fornisce insight statistici sulla relazione tra età e probabilità di sottoscrizione.

In [ ]:
from imblearn.over_sampling import SMOTE
import pandas as pd
import statsmodels.api as sm

# Assuming 'data' is your original DataFrame
X = data[['Age']]  # independent variable
y = data['Subscribed']  # dependent variable

# Apply SMOTE
smote = SMOTE(random_state=42)
X_res, y_res = smote.fit_resample(X, y)

# Convert the resampled data back into a DataFrame
data_resampled = pd.DataFrame(X_res, columns=['Age'])
data_resampled['Subscribed'] = y_res

# Add an intercept
data_resampled['Intercept'] = 1

# Define the model
independent_vars_resampled = ['Intercept', 'Age']
logit_model_resampled = sm.Logit(data_resampled['Subscribed'], data_resampled[independent_vars_resampled])

# Fit the model
fit_resampled = logit_model_resampled.fit()

# Print the summary
print(fit_resampled.summary())

Optimization terminated successfully.
         Current function value: 0.588814
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:             Subscribed   No. Observations:                   62
Model:                          Logit   Df Residuals:                       60
Method:                           MLE   Df Model:                            1
Date:                Tue, 09 Apr 2024   Pseudo R-squ.:                  0.1505
Time:                        14:55:54   Log-Likelihood:                -36.506
converged:                       True   LL-Null:                       -42.975
Covariance Type:            nonrobust   LLR p-value:                 0.0003221
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -5.1668      1.732     -2.983      0.003      -8.562      -1.771
Age            0.1476      0.

In [ ]:
new_class_distribution = y_res.value_counts()
new_class_distribution_percentage = y_res.value_counts(normalize=True) * 100
new_class_distribution, new_class_distribution_percentage

(Subscribed
 0    31
 1    31
 Name: count, dtype: int64,
 Subscribed
 0    50.0
 1    50.0
 Name: proportion, dtype: float64)